# Import potrebných knižníc

In [ ]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from keras.utils import to_categorical
from sklearn.metrics import classification_report, confusion_matrix
from keras.preprocessing.image import ImageDataGenerator

from keras.models import load_model
import shutil

# Klasifkácia BU

Kvôli triedeniu obrázkov BU sme natrénovali klasifikačný model

Úlohou bolo klasifikovanie dát BU vkôli zisteniu úseku s najviac jasnými nocami

Išlo o pomôcku pri získavaní vhodných dát pre ďalšiu prácu ktorá bola neskôr doplnená aj ručnou klasifikáciou

In [ ]:
#klasifikácia do 3 tried pre získanie snímkov s jasnou oblohou
IMG_SIZE = 64
NUM_CLASSES = 3

#definovanie súboru s ručne klasifikovanými obrázkami
DATA_DIR = 'classification/'
CATEGORIES = ['clear', 'clouds', 'half']

#výber triedy s najmenším počtom aby sme mali vyrovnaný počet snímkov v každej triede
min_images = float('inf')
#iterácia cez každú triedu
for category in CATEGORIES:
    path = os.path.join(DATA_DIR, category)
    num_images = len(os.listdir(path))
    #zápis najmenšieho počtu obrázkov
    if num_images < min_images:
        min_images = num_images

X = []
y = []
#výber obrázkov z každej kategórie v počte najmenšej množiny obrázkov
for category in CATEGORIES:
    path = os.path.join(DATA_DIR, category)
    #určenie čísla kategórie podľa priečinka
    class_num = CATEGORIES.index(category)
    images = os.listdir(path)
    #obrázky sú vyberané náhodne
    np.random.shuffle(images)
    num_images = min(min_images, len(images))
    for img in images[:num_images]:
        img_array = np.array(Image.open(os.path.join(path, img)).resize((IMG_SIZE, IMG_SIZE)))
        X.append(img_array)
        y.append(class_num)

X = np.array(X)
y = np.array(y)

#vytvorienie trénovacej a testovacej množiny
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

X_train = X_train / 255.0
X_test = X_test / 255.0

y_train = to_categorical(y_train, NUM_CLASSES)
y_test = to_categorical(y_test, NUM_CLASSES)

#pridanie otočenia obrázkov
datagen = ImageDataGenerator(horizontal_flip=True)

datagen.fit(X_train)

#nastavenie vrstiev modelu
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=X_train.shape[1:]))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(NUM_CLASSES, activation='softmax'))

#nastavenie optimázera
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

#trénovanie modelu na 50 epoch
model.fit(datagen.flow(X_train, y_train, batch_size=32),
                    epochs=50,
                    validation_data=(X_test, y_test))
#výpis accuracy
test_loss, test_acc = model.evaluate(X_test, y_test)
print('Test accuracy:', test_acc)

#testovanie modelu
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)

#vytvorenie metrík modelu
cr = classification_report(np.argmax(y_test, axis=1), y_pred)
print('Classification Report' , cr)

cm = confusion_matrix(np.argmax(y_test, axis=1), y_pred)
print('Confusion matrix:\n', cm)
#uloženie modelu
model.save('model.h5')

Uložený model bol následne používaný na klasifkáciu všetkých zvyšných obrázkov

Následne bola použitá aj ručná klasifikácia

In [ ]:
#načítanie uloženého modelu
model = load_model('model.h5')

#veľkosť obrázkov a počet tried
IMG_SIZE = 64
NUM_CLASSES = 3

#snímky ktoré budú klasifikované
DATA_DIR = '2021_small/'

predictions = []
file_names = []

#vytvorenie priečinkov pre každú triedu
for i in range(NUM_CLASSES):
    class_dir = os.path.join('rio_grande2021_crop', str(i))
    os.makedirs(class_dir, exist_ok=True)

#iterácia pre každý snímok
for item in os.listdir(DATA_DIR):
    item_path = os.path.join(DATA_DIR, item)
    
    #kontrola či ide o súbor, poistka kvôli chybným súborom prípadne ak by sa do priečinka dostali iné súbory
    if os.path.isfile(item_path):
        #načítanie snímku a prevedenie na správnu veľkosť
        img_array = np.array(Image.open(item_path).resize((IMG_SIZE, IMG_SIZE)))
        
        #normalizácia
        img_array = img_array / 255.0
        
        #pridanie dimenzie
        img_array = np.expand_dims(img_array, axis=0)
        img_array = np.expand_dims(img_array, axis=-1)
        
        #predikcia
        prediction = model.predict(img_array)
        prediction = np.argmax(prediction)
        
        #pridanie obrázkov do listov
        predictions.append(prediction)
        file_names.append(item)
        
        #presun do súboru s výsledkami
        src_path = item_path
        dst_dir = os.path.join('rio_grande2021_crop', str(prediction))
        dst_path = os.path.join(dst_dir, item)
        shutil.move(src_path, dst_path)
